# Классические нейронные сети

In [ ]:
#pip install pymorphy3
#pip install tensorflow==2.17.0

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from pymorphy3 import MorphAnalyzer

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Bidirectional, Dense

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.metrics import cohen_kappa_score

In [ ]:
df=pd.read_excel('marked_data.xlsx')
df['labels'] = df['labels'] + 1

# **Preprosessing**

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
morph = MorphAnalyzer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def text_preprocessing (text):
    soup = BeautifulSoup(text, "html.parser")
    clean_text = soup.get_text() #чтобы изначально удалить html-теги
    clean_text = re.sub(r'[^А-я\s]',' ',clean_text) #удаление неалфавитных символов (в том числе знаков пунктуации)
    clean_text = clean_text.lower() #приведение к нижнему регистру
    word_tokens = [morph.normal_forms(word)[0] for word in clean_text.split() if word not in stop_words] #удаление стоп-слов и лемматизация
    clean_text = ' '.join(word_tokens)
    return clean_text

In [ ]:
df["clean_text"] = df["text"].map(text_preprocessing)

# **Vectorization**

In [ ]:
df_train = df[:7000]
df_val = df[7000:8000]
df_test = df[8000:]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train.clean_text)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
print(vocab_size)

13901


In [ ]:
len_comments = pd.DataFrame(list(map(lambda x: len(x.split()), df.clean_text)))

In [ ]:
len_comments.median()[0]

6.0

In [ ]:
print((len_comments < 30).mean()[0])

0.9503


In [ ]:
print((len_comments < 100).mean()[0])

0.9949


In [ ]:
x_train = pad_sequences(tokenizer.texts_to_sequences(df_train.clean_text),maxlen = 30)
x_val = pad_sequences(tokenizer.texts_to_sequences(df_val.clean_text),maxlen = 30)
x_test = pad_sequences(tokenizer.texts_to_sequences(df_test.clean_text),maxlen = 30)

In [ ]:
encoder = LabelEncoder()
encoder.fit(df_train.labels.to_list())
y_train = encoder.transform(df_train.labels.to_list())
y_val = encoder.transform(df_val.labels.to_list())
y_test = encoder.transform(df_test.labels.to_list())
y_train = y_train.reshape(-1,1)
y_val = y_val.reshape(-1,1)
y_test = y_test.reshape(-1,1)

## **GRU**

In [ ]:
embedding_dim = 32
model_GRU = Sequential()
model_GRU.add(Embedding(vocab_size, embedding_dim, input_length = 200))
model_GRU.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
model_GRU.add(Dense(3, activation='softmax'))

In [ ]:
model_GRU.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Обучение модели
model_GRU.fit(x_train, y_train, epochs=2, batch_size=16, validation_data=(x_val, y_val))
loss, accuracy = model_GRU.evaluate(x_val, y_val)
print(f'Accuracy: {accuracy:.2f}')

Epoch 1/2
438/438 ━━━━━━━━━━━━━━━━━━━━ 25s 47ms/step - accuracy: 0.6406 - loss: 0.8051 - val_accuracy: 0.5490 - val_loss: 1.0110
Epoch 2/2
438/438 ━━━━━━━━━━━━━━━━━━━━ 39s 43ms/step - accuracy: 0.7848 - loss: 0.5233 - val_accuracy: 0.5380 - val_loss: 1.2136
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5482 - loss: 1.1878
Accuracy: 0.54


In [ ]:
print(model_GRU.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 30, 32)              │         444,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ (None, 128)                 │          62,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,522,283 (5.81 MB)

 Trainable params: 507,427 (1.94 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,014,856 (3.87 MB)

None


In [ ]:
predict_test = model_GRU.predict(x_test, verbose=0)
df_test_GRU = df_test.copy()
df_test_GRU['predict'] = predict_test.argmax(1).tolist()
df_test_GRU['Pneg'] = predict_test[:,0]
df_test_GRU['Pneutral'] = predict_test[:,1]
df_test_GRU['Ppos'] = predict_test[:,2]
df_test_GRU['Ppos-Pneg'] = df_test_GRU['Ppos'] - df_test_GRU['Pneg']

In [ ]:
print('accuracy', accuracy_score(df_test_GRU['labels'], df_test_GRU['predict']))
print('macro_precision', precision_score(df_test_GRU['labels'], df_test_GRU['predict'], average='macro'))
print('macro_recall', recall_score(df_test_GRU['labels'], df_test_GRU['predict'], average='macro'))
print('macro_f1', f1_score(df_test_GRU['labels'], df_test_GRU['predict'], average='macro'))
print('RMSE for Ppos-Pneg', root_mean_squared_error(df_test_GRU['labels']-1, df_test_GRU['Ppos-Pneg']))
print('MAE for Ppos-Pneg', mean_absolute_error(df_test_GRU['labels']-1, df_test_GRU['Ppos-Pneg']))

accuracy 0.532
macro_precision 0.4891426008664343
macro_recall 0.48380306413992
macro_f1 0.4855911483870467
RMSE for Ppos-Pneg 0.7953473329544067
MAE for Ppos-Pneg 0.5971473455429077


In [ ]:
print('precision by class', precision_score(df_test_GRU['labels'], df_test_GRU['predict'], average=None))
print('recall by class', recall_score(df_test_GRU['labels'], df_test_GRU['predict'], average=None))
print('f1 by class', f1_score(df_test_GRU['labels'], df_test_GRU['predict'], average=None))

precision by class [0.63376111 0.45131376 0.38235294]
recall by class [0.65310275 0.47249191 0.32581454]
f1 by class [0.64328657 0.46166008 0.35182679]


In [ ]:
confusion_matrix(df_test_GRU['labels'], df_test_GRU['predict'], labels=[0,1,2])

array([[642, 238, 103],
       [219, 292, 107],
       [152, 117, 130]])

In [ ]:
cohen_kappa_score(df_test_GRU['labels'], df_test_GRU['predict'], labels=None, weights= 'quadratic', sample_weight=None)

0.2736877702855034

In [ ]:
df_test_GRU.to_excel('Test_GRU.xlsx')

## **LSTM**

In [ ]:
embedding_dim = 32
model_LSTM = Sequential()
model_LSTM.add(Embedding(vocab_size, embedding_dim, input_length = 200))
model_LSTM.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model_LSTM.add(Dense(3, activation='softmax'))

In [ ]:
model_LSTM.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Обучение модели
model_LSTM.fit(x_train, y_train, epochs=2, batch_size=16, validation_data=(x_val, y_val))
loss, accuracy = model_LSTM.evaluate(x_val, y_val)
print(f'Accuracy: {accuracy:.2f}')

Epoch 1/2
438/438 ━━━━━━━━━━━━━━━━━━━━ 26s 52ms/step - accuracy: 0.7876 - loss: 0.5288 - val_accuracy: 0.5240 - val_loss: 1.1180
Epoch 2/2
438/438 ━━━━━━━━━━━━━━━━━━━━ 22s 50ms/step - accuracy: 0.8533 - loss: 0.3808 - val_accuracy: 0.5290 - val_loss: 1.3103
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5295 - loss: 1.3002
Accuracy: 0.53


In [ ]:
print(model_LSTM.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 30, 32)              │         444,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │          82,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,582,955 (6.04 MB)

 Trainable params: 527,651 (2.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,055,304 (4.03 MB)

None


In [ ]:
predict_test = model_LSTM.predict(x_test, verbose=0)
df_test_LSTM = df_test.copy()
df_test_LSTM['predict'] = predict_test.argmax(1).tolist()
df_test_LSTM['Pneg'] = predict_test[:,0]
df_test_LSTM['Pneutral'] = predict_test[:,1]
df_test_LSTM['Ppos'] = predict_test[:,2]
df_test_LSTM['Ppos-Pneg'] = df_test_LSTM['Ppos'] - df_test_LSTM['Pneg']

In [ ]:
print('accuracy', accuracy_score(df_test_LSTM['labels'], df_test_LSTM['predict']))
print('macro_precision', precision_score(df_test_LSTM['labels'], df_test_LSTM['predict'], average='macro'))
print('macro_recall', recall_score(df_test_LSTM['labels'], df_test_LSTM['predict'], average='macro'))
print('macro_f1', f1_score(df_test_LSTM['labels'], df_test_LSTM['predict'], average='macro'))
print('RMSE for Ppos-Pneg', root_mean_squared_error(df_test_LSTM['labels']-1, df_test_LSTM['Ppos-Pneg']))
print('MAE for Ppos-Pneg', mean_absolute_error(df_test_LSTM['labels']-1, df_test_LSTM['Ppos-Pneg']))

accuracy 0.537
macro_precision 0.4896299390723107
macro_recall 0.4670013717474095
macro_f1 0.4700105915861204
RMSE for Ppos-Pneg 0.8050621151924133
MAE for Ppos-Pneg 0.5858951807022095


In [ ]:
print('precision by class', precision_score(df_test_LSTM['labels'], df_test_LSTM['predict'], average=None))
print('recall by class', recall_score(df_test_LSTM['labels'], df_test_LSTM['predict'], average=None))
print('f1 by class', f1_score(df_test_LSTM['labels'], df_test_LSTM['predict'], average=None))

precision by class [0.60479042 0.45500849 0.40909091]
recall by class [0.71922686 0.43365696 0.2481203 ]
f1 by class [0.6570632  0.44407622 0.30889236]


In [ ]:
confusion_matrix(df_test_LSTM['labels'], df_test_LSTM['predict'], labels=[0,1,2])

array([[707, 205,  71],
       [278, 268,  72],
       [184, 116,  99]])

In [ ]:
cohen_kappa_score(df_test_LSTM['labels'], df_test_LSTM['predict'], labels=None, weights= 'quadratic', sample_weight=None)

0.24898917762760508

In [ ]:
df_test_LSTM.to_excel('Test_LSTM.xlsx')

## CNN

In [ ]:
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D

In [ ]:
embedding_dim = 32
model_CNN = Sequential()
model_CNN.add(Embedding(vocab_size, embedding_dim, input_length = 200))
model_CNN.add(Conv1D(filters=128, kernel_size=3, activation='relu')),
model_CNN.add(GlobalMaxPooling1D()),
model_CNN.add(Dense(3, activation='softmax'))

In [ ]:
model_CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # Обучение модели
model_CNN.fit(x_train, y_train, epochs=2, batch_size=16, validation_data=(x_val, y_val))
loss, accuracy = model_CNN.evaluate(x_val, y_val)
print(f'Accuracy: {accuracy:.2f}')

Epoch 1/2
438/438 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.5042 - loss: 1.0133 - val_accuracy: 0.5470 - val_loss: 0.9565
Epoch 2/2
438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6488 - loss: 0.8134 - val_accuracy: 0.5630 - val_loss: 0.9439
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5470 - loss: 0.9593
Accuracy: 0.56


In [ ]:
print(model_CNN.summary())

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 30, 32)              │         444,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 28, 128)             │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 128)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,372,907 (5.24 MB)

 Trainable params: 457,635 (1.75 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 915,272 (3.49 MB)

None


In [ ]:
predict_test = model_CNN.predict(x_test, verbose=0)
df_test_CNN = df_test.copy()
df_test_CNN['predict'] = predict_test.argmax(1).tolist()
df_test_CNN['Pneg'] = predict_test[:,0]
df_test_CNN['Pneutral'] = predict_test[:,1]
df_test_CNN['Ppos'] = predict_test[:,2]
df_test_CNN['Ppos-Pneg'] = df_test_CNN['Ppos'] - df_test_CNN['Pneg']

In [ ]:
print('accuracy', accuracy_score(df_test_CNN['labels'], df_test_CNN['predict']))
print('macro_precision', precision_score(df_test_CNN['labels'], df_test_CNN['predict'], average='macro'))
print('macro_recall', recall_score(df_test_CNN['labels'], df_test_CNN['predict'], average='macro'))
print('macro_f1', f1_score(df_test_CNN['labels'], df_test_CNN['predict'], average='macro'))
print('RMSE for Ppos-Pneg', root_mean_squared_error(df_test_CNN['labels']-1, df_test_CNN['Ppos-Pneg']))
print('MAE for Ppos-Pneg', mean_absolute_error(df_test_CNN['labels']-1, df_test_CNN['Ppos-Pneg']))

accuracy 0.5515
macro_precision 0.516973278102664
macro_recall 0.480066137683029
macro_f1 0.4747059124989174
RMSE for Ppos-Pneg 0.7388778328895569
MAE for Ppos-Pneg 0.5910289287567139


In [ ]:
print('precision by class', precision_score(df_test_CNN['labels'], df_test_CNN['predict'], average=None))
print('recall by class', recall_score(df_test_CNN['labels'], df_test_CNN['predict'], average=None))
print('f1 by class', f1_score(df_test_CNN['labels'], df_test_CNN['predict'], average=None))

precision by class [0.62685185 0.46354167 0.46052632]
recall by class [0.68870804 0.57605178 0.1754386 ]
f1 by class [0.65632574 0.51370851 0.25408348]


In [ ]:
confusion_matrix(df_test_CNN['labels'], df_test_CNN['predict'], labels=[0,1,2])

array([[677, 263,  43],
       [223, 356,  39],
       [180, 149,  70]])

In [ ]:
cohen_kappa_score(df_test_CNN['labels'], df_test_CNN['predict'], labels=None, weights= 'quadratic', sample_weight=None)

0.24422600248642878

In [ ]:
df_test_CNN.to_excel('Test_CNN.xlsx')

# Предсказания моделей на данных по всем комментариям

In [ ]:
df_ec = pd.read_csv('ec_comments.csv')
df_cb = pd.read_csv('comments_with_cb.csv')

In [ ]:
df_ec["clean_text"] = df_ec["text"].map(text_preprocessing)
df_cb["clean_text"] = df_cb["text"].map(text_preprocessing)

In [ ]:
ec_vec = pad_sequences(tokenizer.texts_to_sequences(df_ec.clean_text),maxlen = 30)
cb_vec = pad_sequences(tokenizer.texts_to_sequences(df_cb.clean_text),maxlen = 30)

In [ ]:
models = {'GRU': model_GRU, 'LSTM': model_LSTM, 'CNN': model_CNN}

In [ ]:
for model_name in models.keys():
    m = models[model_name]
    ec = df_ec.copy()
    cb = df_cb.copy()

    predict_ec = m.predict(ec_vec)
    ec['predict'] = predict_ec.argmax(1).tolist()
    ec['Pneg'] = predict_ec[:,0]
    ec['Pneutral'] = predict_ec[:,1]
    ec['Ppos'] = predict_ec[:,2]
    ec['Ppos-Pneg'] = ec['Ppos'] - ec['Pneg']
    path = f'ec_{model_name}.csv'
    ec.to_csv(path)

    predict_cb = m.predict(cb_vec)
    cb['predict'] = predict_cb.argmax(1).tolist()
    cb['Pneg'] = predict_cb[:,0]
    cb['Pneutral'] = predict_cb[:,1]
    cb['Ppos'] = predict_cb[:,2]
    cb['Ppos-Pneg'] = cb['Ppos'] - cb['Pneg']
    path = f'cb_{model_name}.csv'
    cb.to_csv(path)

    print(model_name)

6265/6265 ━━━━━━━━━━━━━━━━━━━━ 42s 7ms/step
2330/2330 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step
GRU
6265/6265 ━━━━━━━━━━━━━━━━━━━━ 47s 8ms/step
2330/2330 ━━━━━━━━━━━━━━━━━━━━ 18s 8ms/step
LSTM
6265/6265 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step
2330/2330 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
CNN
